In [1]:
%matplotlib inline
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import sys
import os
from pathlib import Path

os.chdir(path="/Research")
print(Path.cwd())
sys.path.append(os.path.abspath('/Research/gpudrive'))

from gpudrive.env.config import EnvConfig
from gpudrive.env.env_torch import GPUDriveTorchEnv
from gpudrive.env.dataset import SceneDataLoader
import importlib

from CMTR.mtr.config import cfg_from_list
# from CMTR.mtr.datasets.dataset import DatasetTemplate
from CMTR.mtr.datasets.waymo.waymo_dataset import WaymoDataset
# from CMTR.mtr.models.context_encoder import MTREncoder
import yaml 
from easydict import EasyDict
import logging
import torch
import numpy as np

# import CMTR.mtr.datasets.waymo.waymo_dataset
# importlib.reload(CMTR.mtr.datasets.waymo.waymo_dataset)
# from CMTR.mtr.datasets.waymo.waymo_dataset import WaymoDataset

/Research


In [2]:

DYNAMICS_MODEL = "delta_local" # "delta_local" / "state" / "classic"
DATA_PATH = "data/processed/examples" # Your data path
MAX_NUM_OBJECTS = 64
NUM_ENVS = 2
DEVICE = "cuda:0"

# Configs
env_config = EnvConfig(dynamics_model=DYNAMICS_MODEL)

In [3]:
# Make dataloader
data_loader = SceneDataLoader(
    root="gpudrive/data/processed/examples", # Path to the dataset
    batch_size=NUM_ENVS, # Batch size, you want this to be equal to the number of worlds (envs) so that every world receives a different scene
    dataset_size=NUM_ENVS, # Total number of different scenes we want to use
    sample_with_replacement=False, 
    seed=42, 
    shuffle=True,   
)

print("Making the environment")

# Make environment
env = GPUDriveTorchEnv(
    config=env_config,
    data_loader=data_loader,
    max_cont_agents=MAX_NUM_OBJECTS, # Maximum number of agents to control per scenario
    device="cuda", 
    action_type="continuous" # "continuous" or "discrete"
)

Making the environment
Compiling GPU engine code:
/Research/gpudrive/external/madrona/src/mw/device/memory.cpp
/Research/gpudrive/external/madrona/src/mw/device/state.cpp
/Research/gpudrive/external/madrona/src/mw/device/crash.cpp
/Research/gpudrive/external/madrona/src/mw/device/consts.cpp
/Research/gpudrive/external/madrona/src/mw/device/taskgraph.cpp
/Research/gpudrive/external/madrona/src/mw/device/taskgraph_utils.cpp
/Research/gpudrive/external/madrona/src/mw/device/sort_archetype.cpp
/Research/gpudrive/external/madrona/src/mw/device/host_print.cpp
/Research/gpudrive/external/madrona/src/mw/../common/hashmap.cpp
/Research/gpudrive/external/madrona/src/mw/../common/navmesh.cpp
/Research/gpudrive/external/madrona/src/mw/../core/base.cpp
/Research/gpudrive/external/madrona/src/mw/../physics/physics.cpp
/Research/gpudrive/external/madrona/src/mw/../physics/geo.cpp
/Research/gpudrive/external/madrona/src/mw/../physics/xpbd.cpp
/Research/gpudrive/external/madrona/src/mw/../physics/tgs.c

/Research/gpudrive/src/level_gen.cpp(283): warning #177-D: function "madrona_gpudrive::createFloorPlane" was declared but never referenced
  static void createFloorPlane(Engine &ctx)
              ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"





In [6]:
import dev
import dev.imitation_data_generation

importlib.reload(dev)
importlib.reload(dev.imitation_data_generation)

from dev.imitation_data_generation import generate_state_action_pairs


import CMTR
import CMTR.mtr
import CMTR.mtr.datasets
import CMTR.mtr.datasets.waymo.waymo_dataset

importlib.reload(CMTR)
importlib.reload(CMTR.mtr)
importlib.reload(CMTR.mtr.datasets)
importlib.reload(CMTR.mtr.datasets.waymo.waymo_dataset)

from CMTR.mtr.datasets.waymo.waymo_dataset import WaymoDataset



In [7]:
'''
Generating state-action pairs: Part of the dataloader
'''
import logging

logger = logging.getLogger(__name__)

def getconfig(cfg_file):
    with open(cfg_file, 'r') as f:
        try:
            config = yaml.load(f, Loader=yaml.FullLoader)
        except:
            config = yaml.load(f)

    return EasyDict(config)

logger = logging.getLogger(__name__)

config = getconfig("dev/configs/config.yaml")
waymo_dataset = WaymoDataset(config.DATA_CONFIG, test_mode=1, logger=logger)

infos = generate_state_action_pairs(env=env,
                            device="cuda",
                            action_space_type="continuous", 
                            use_action_indices=True,  
                            make_video=False,
                            render_index=[2, 0],
                            save_path="use_discr_actions_fix",
                            CMTR=True, 
                            waymo_dataset=waymo_dataset,
                            prior_frame=40,
                            future_frame=40
) 

torch.Size([47, 91, 12]) torch.Size([64, 91, 2])


RuntimeError: Sizes of tensors must match except in dimension 2. Expected size 47 but got size 64 for tensor number 1 in the list.

In [50]:
infos['map_infos']['localized_polylines'].shape

torch.Size([8, 200, 1, 13])

In [54]:
infos['map_infos']['localized_polylines_mask'].shape

torch.Size([8, 200, 1])

In [56]:
infos['map_infos']['localized_polylines_center'].shape

torch.Size([8, 200, 3])

In [61]:
expert_actions, expert_speeds, expert_positions, expert_yaws = env.get_expert_actions()

In [62]:
expert_actions.shape

torch.Size([2, 64, 91, 3])

In [68]:
infos[0][0]['obj_trajs'].shape

(8, 47, 11, 29)

In [69]:
infos[0][0]['obj_trajs_future_state'].shape

(8, 47, 80, 4)

In [8]:
waymo_dataset = WaymoDataset(config.DATA_CONFIG, test_mode=1, logger=logger)

In [9]:
ret_dict = waymo_dataset.create_scene_level_data(0)

In [10]:
obj_trajs = ret_dict['obj_trajs']

In [11]:
obj_trajs.shape # (num_center_objects, num_objects, num_timestamps, num_attrs)

(3, 22, 11, 29)

In [13]:
from gpudrive.datatypes.observation import (
    LocalEgoState,
    GlobalEgoState,
    # PartnerObs,
)

global_ego = GlobalEgoState.from_tensor(
    abs_self_obs_tensor=env.sim.absolute_self_observation_tensor(),
    backend=env.backend,
    device="cuda"
)

In [18]:
control_mask = env.get_controlled_agents_mask()
goals_x = global_ego.goal_x

In [26]:
list(zip(goals_x[~control_mask], global_ego.pos_x[~control_mask]))

[(tensor(-6.7849, device='cuda:0'), tensor(-6.7849, device='cuda:0')),
 (tensor(-2.4020, device='cuda:0'), tensor(-2.4020, device='cuda:0')),
 (tensor(7.9561, device='cuda:0'), tensor(7.9561, device='cuda:0')),
 (tensor(-23.5463, device='cuda:0'), tensor(-23.5463, device='cuda:0')),
 (tensor(-36.5822, device='cuda:0'), tensor(-36.5822, device='cuda:0')),
 (tensor(-12.2128, device='cuda:0'), tensor(-12.2128, device='cuda:0')),
 (tensor(-11.7589, device='cuda:0'), tensor(-11.7589, device='cuda:0')),
 (tensor(3.9974, device='cuda:0'), tensor(3.9974, device='cuda:0')),
 (tensor(-47.6550, device='cuda:0'), tensor(-47.6550, device='cuda:0')),
 (tensor(13.9465, device='cuda:0'), tensor(13.9465, device='cuda:0')),
 (tensor(-16.8408, device='cuda:0'), tensor(-16.8408, device='cuda:0')),
 (tensor(-14.7782, device='cuda:0'), tensor(-14.7782, device='cuda:0')),
 (tensor(-42.1628, device='cuda:0'), tensor(-42.1628, device='cuda:0')),
 (tensor(-25.2578, device='cuda:0'), tensor(-25.2578, device='cud

In [27]:
list(zip(goals_x[control_mask], global_ego.pos_x[control_mask]))

[(tensor(22.0682, device='cuda:0'), tensor(11.9054, device='cuda:0')),
 (tensor(2.2360, device='cuda:0'), tensor(-21.0901, device='cuda:0')),
 (tensor(-63.2372, device='cuda:0'), tensor(-41.3113, device='cuda:0')),
 (tensor(70.2335, device='cuda:0'), tensor(38.1151, device='cuda:0')),
 (tensor(-15.3058, device='cuda:0'), tensor(-48.5161, device='cuda:0')),
 (tensor(38.7407, device='cuda:0'), tensor(39.3571, device='cuda:0')),
 (tensor(30.9185, device='cuda:0'), tensor(25.8573, device='cuda:0')),
 (tensor(12.3284, device='cuda:0'), tensor(-7.4191, device='cuda:0')),
 (tensor(95.7842, device='cuda:0'), tensor(40.6240, device='cuda:0')),
 (tensor(93.8916, device='cuda:0'), tensor(22.0947, device='cuda:0')),
 (tensor(-39.9854, device='cuda:0'), tensor(29.4111, device='cuda:0')),
 (tensor(99.8877, device='cuda:0'), tensor(83.8340, device='cuda:0')),
 (tensor(26.5371, device='cuda:0'), tensor(39.0342, device='cuda:0'))]

In [42]:
import dev
import dev.imitation_data_generation

importlib.reload(dev)
importlib.reload(dev.imitation_data_generation)

from dev.imitation_data_generation import generate_state_action_pairs

In [43]:
'''
Generating state-action pairs: Part of the dataloader
'''
waymo_dataset = WaymoDataset(config.DATA_CONFIG, test_mode=1, logger=logger)
infos = generate_state_action_pairs(env=env,
                            device="cuda",
                            action_space_type="continuous", 
                            use_action_indices=True,  
                            make_video=False,
                            render_index=[2, 0],
                            save_path="use_discr_actions_fix",
                            CMTR=True, 
                            waymo_dataset=waymo_dataset,
                            prior_frame=40,
                            future_frame=40
) 

In [35]:
ret_dict = infos[0][0]

In [36]:
ret_dict['center_gt_trajs'].shape

(8, 80, 4)

In [37]:
ret_dict['obj_trajs'].shape

(8, 47, 11, 31)